### Импортируем библиотеки и загружаем данные

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv
pd.set_option('display.max_colwidth', None)

### Загружаем файлы и сводим в один

In [41]:
df = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\Пыпин\01.2026\realty_sold_15012026_M.xlsx", sheet_name="Данные")

In [42]:
df2 = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\Пыпин\01.2026\realty_sold_150125_NM.xlsx", sheet_name="Данные")

In [43]:
df3 = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\Пыпин\01.2026\realty_sold_15012026_MO.xlsx", sheet_name="Данные")

In [70]:
df_общий = pd.concat([df, df2, df3], ignore_index=True)

In [69]:
df_общий.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2003181 entries, 0 to 2075912
Data columns (total 20 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   ID ЖК                    float64       
 1   ЖК рус                   object        
 2   Район Город              object        
 3   Округ Направление        object        
 4   Регион                   object        
 5   АТД                      object        
 6   Застройщик ЖК            object        
 7   Площадь                  float64       
 8   Комнатность              float64       
 9   Тип Комнатности          object        
 10  Тип помещения            object        
 11  Корпус                   object        
 12  Покупатель ЮЛ            object        
 13  Дата регистрации модель  datetime64[ns]
 14  Залогодержатель          object        
 15  Тип обременения          object        
 16  Оценка цены              float64       
 17  Купил лотов в ЖК         float64

### Убираем лишние столбцы и выставляем верный порядок

In [64]:
# на всякий случай убираем пробелы
df_общий.columns = df_общий.columns.str.strip()

# нужный порядок столбцов
columns_order = [
    "ID ЖК",
    "ЖК рус",
    "Район Город",
    "Округ Направление",
    "Регион",
    "АТД",
    "Застройщик ЖК",
    "Площадь",
    "Комнатность",
    "Тип Комнатности",
    "Тип помещения",
    "Корпус",
    "Покупатель ЮЛ",
    "Дата регистрации модель",
    "Залогодержатель",
    "Тип обременения",
    "Оценка цены",
    "Купил лотов в ЖК",
    "класс",
    "Ипотека"
]

# оставляем и упорядочиваем столбцы
df_общий = df_общий.reindex(columns=columns_order)

### Проверяем тип данных в столбце 'Дата регистрации модель'. Должен быть один тип данных

In [50]:
df_общий['Дата регистрации модель'].map(type).value_counts()

Дата регистрации модель
<class 'pandas._libs.tslibs.timestamps.Timestamp'>    2003181
Name: count, dtype: int64

### Приводим столбец 'Дата регистрации модель' к единому типу данных 

In [65]:
# Функция для конвертации одного значения
def parse_date(x):
    if pd.isna(x):
        return pd.NaT
    # Если уже Timestamp — оставляем
    if isinstance(x, pd.Timestamp):
        return x
    # Если float — Excel serial date
    elif isinstance(x, (float, int)):
        # Игнорируем маленькие числа (<32), чтобы не было 1970
        if x > 31:
            return pd.to_datetime(x, unit='D', origin='1899-12-30', errors='coerce')
        else:
            return pd.NaT
    # Если строка — пробуем распарсить
    else:
        x = str(x).strip()
        return pd.to_datetime(x, dayfirst=True, errors='coerce')

# Применяем функцию ко всему столбцу
df_общий['Дата регистрации модель'] = df_общий['Дата регистрации модель'].apply(parse_date)

# Убираем строки, где дата не распарсилась
df_общий = df_общий.dropna(subset=['Дата регистрации модель'])

# Проверяем результат
print(df_общий['Дата регистрации модель'].dtype)
print(df_общий['Дата регистрации модель'].head())

datetime64[ns]
0   2025-03-13
1   2023-12-22
2   2023-12-20
3   2023-12-18
4   2023-12-18
Name: Дата регистрации модель, dtype: datetime64[ns]


### Проверяем как выглядят id, если требуется - меняем тип данных на строковой

In [52]:
df_общий['ID ЖК'].unique()

array(['2062', '8647', '8539', ..., '1071', '6961', '6951'],
      shape=(1128,), dtype=object)

In [51]:
df_общий['ID ЖК'] = df_общий['ID ЖК'].astype(str)
df_общий['ID ЖК'] = df_общий['ID ЖК'].str.replace(r'\.0$', '', regex=True)

### Проверим количество строк по годам

In [71]:
# Создаем столбец с годом
df_общий['Год'] = df_общий['Дата регистрации модель'].dt.year

# Считаем количество строк по каждому году
counts_by_year = df_общий.groupby('Год').size().reset_index(name='Количество')

print(counts_by_year)
df_общий = df_общий.drop(columns=['Год'])

       Год  Количество
0   2005.0           2
1   2006.0           1
2   2007.0           3
3   2008.0          70
4   2009.0         165
5   2010.0          87
6   2011.0        2470
7   2012.0        4245
8   2013.0       10003
9   2014.0       25625
10  2015.0       49671
11  2016.0       90424
12  2017.0      127395
13  2018.0      179780
14  2019.0      226842
15  2020.0      211061
16  2021.0      237454
17  2022.0      188261
18  2023.0      237130
19  2024.0      212147
20  2025.0      200100
21  2026.0         245


### Убираем кавычки в текстовых столбцах

In [53]:
df_общий['Покупатель ЮЛ'] = df_общий['Покупатель ЮЛ'].str.replace('""', '"', regex=False)

In [54]:
df_общий['Застройщик ЖК'] = df_общий['Застройщик ЖК'].str.replace('""', '"', regex=False)

In [55]:
df_общий['Залогодержатель'] = df_общий['Залогодержатель'].str.replace('""', '"', regex=False)

In [56]:
df_общий['Покупатель ЮЛ'] = df_общий['Покупатель ЮЛ'].str.replace('"', '', regex=False)

In [57]:
df_общий["Купил лотов в ЖК"] = pd.to_numeric(df_общий["Купил лотов в ЖК"], errors="coerce").astype("Int64")

### Если будет нужно проверить конкретную строку:

In [65]:
row_index = 2132399 # индексация с 0
print(df_общий.iloc[row_index])

ID ЖК                                               6659
ЖК рус                                             Юнино
Район Город                                     Щербинка
Округ Направление                                    НАО
Регион                                      Новая Москва
АТД                                                  НАО
Застройщик ЖК                                        ПИК
Площадь                                             32.0
Комнатность                                          nan
Тип Комнатности                                        1
Тип помещения                                   квартира
Корпус                                        корпус 1.1
Покупатель ЮЛ                    ООО "ПИК-ИИНВЕСТПРОЕКТ"
Дата регистрации модель    1970-01-01 00:00:00.000044939
Залогодержатель                                      nan
Тип обременения                                      nan
Оценка цены                                    6902784.0
Купил лотов в ЖК               

### Если будут проблемы, то ищем и проверяем что содержат нужные строки. 

In [18]:
filtered_df = df_общий[df_общий['Район Город'].str.contains('Дрожжино', na=False)]
filtered_df

,ID ЖК,ЖК рус,Район Город,Округ Направление,Регион,АТД,Застройщик ЖК,Площадь,Комнатность,Тип Комнатности,Тип помещения,Корпус,Покупатель ЮЛ,Дата регистрации модель,Залогодержатель,Тип обременения,Оценка цены,Купил лотов в ЖК,класс,Ипотека
965179,1007,Бутово Парк 1,Дрожжино (п.г.т.),Симферопольское шоссе,Московская область,Ленинский городской округ,Мортон (ПИК),59.6,1.0,1,квартира,корпус 4а,NaN,2019-09-06,NaN,NaN,9029400.0,1,эконом,0.0
965180,1007,Бутово Парк 1,Дрожжино (п.г.т.),Симферопольское шоссе,Московская область,Ленинский городской округ,Мортон (ПИК),20.4,1.0,ст,квартира,корпус 4а,NaN,2019-06-20,NaN,NaN,3090600.0,1,эконом,0.0
965181,1007,Бутово Парк 1,Дрожжино (п.г.т.),Симферопольское шоссе,Московская область,Ленинский городской округ,Мортон (ПИК),56.9,NaN,2,квартира,корпус 4а,NaN,2019-06-17,ВТБ,ипотека,7493730.0,1,эконом,1.0
965182,1007,Бутово Парк 1,Дрожжино (п.г.т.),Симферопольское шоссе,Московская область,Ленинский городской округ,Мортон (ПИК),91.0,3.0,3,квартира,корпус 4а,NaN,2019-06-06,NaN,NaN,9982700.0,1,эконом,0.0
965183,1007,Бутово Парк 1,Дрожжино (п.г.т.),Симферопольское шоссе,Московская область,Ленинский городской округ,Мортон (ПИК),90.3,NaN,3,квартира,корпус 4а,NaN,2019-06-06,СБЕРБАНК,ипотека,9698220.0,1,эконом,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1790808,4622,Южная Битца,Дрожжино (п.г.т.),Симферопольское шоссе,Московская область,Ленинский городской округ,ДСК-1 (ФСК Лидер),24.7,1.0,ст,квартира,корпус 10,NaN,2023-11-01,NaN,NaN,5862298.0,2,эконом (панель),0.0
1790809,4622,Южная Битца,Дрожжино (п.г.т.),Симферопольское шоссе,Московская область,Ленинский городской округ,ДСК-1 (ФСК Лидер),43.8,NaN,2,квартира,корпус 10,NaN,2023-10-30,СБЕРБАНК,ипотека,9246180.0,2,эконом (панель),1.0
1790810,4622,Южная Битца,Дрожжино (п.г.т.),Симферопольское шоссе,Московская область,Ленинский городской округ,ДСК-1 (ФСК Лидер),37.6,NaN,1,квартира,корпус 10,NaN,2023-10-30,СБЕРБАНК,ипотека,8923984.0,1,эконом (панель),1.0
1790811,4622,Южная Битца,Дрожжино (п.г.т.),Симферопольское шоссе,Московская область,Ленинский городской округ,ДСК-1 (ФСК Лидер),14.4,NaN,NaN,машино-место,корпус 30 (Паркинг),ООО Западная региональная девелоперская компания,2023-10-25,NaN,NaN,1159560.0,19,эконом (панель),0.0


### Сохраняем в файл после всех манипуляций

In [58]:
df_общий.to_csv(r"C:\Users\m.olshanskiy\Desktop\Pipin-01-2026.csv", index=False, encoding='utf-8-sig')